In [1]:
from time import time

import sys
sys.path.append('..')
import util

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

log_board = util.diagnostics.LogBoard('log_dir', 6005)
log_board.launch()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
patch_size = 128
train = util.data.SenNet(
    patch_size,
    guarantee_vessel=0.5,
    data=(
        [
            torch.cat([
                torch.load('/root/data/model_outputs/kidney_3_sparse_3d_raw_mask.pt'),
                torch.load('/root/data/cache/train/kidney_3_sparse/images.pt'),
            ])
        ],
        [torch.load('/root/data/cache/train/kidney_3_sparse/labels.pt')],
    )
)

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6005/ (Press CTRL+C to quit)


In [3]:
# class _FilterNet(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         # assert len(layers) >= 2, "layers must have at least 2 elements"

#         self.kernel_size = 17
#         self.closing_size = (self.kernel_size - 1) // 2
#         nlayers = 3

#         self.layers = nn.ModuleList()
#         self.layers.append(util.FFTConv3d(2, 2, self.kernel_size))
#         for _ in range(nlayers):
#             self.layers.append(util.FFTConv3d(4, 2, self.kernel_size))
#         # self.layers.append(
#         #     util.FFTConv3d(2, 1, kernel_size),
#         #     # torch.nn.BatchNorm3d(layers[-1]),
#         # )
#         self.nonlinearity_norm = nn.Sequential(
#             torch.nn.BatchNorm3d(1),
#             torch.nn.GELU(),
#         )
    
#         # self.net = torch.nn.Sequential(*L)
    
#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         z = self.layers[0](x)
#         agg_z = [z]
#         for i, layer in enumerate(self.layers[1:]):
#             z = layer(
#                 torch.cat([
#                     (F.sigmoid(agg_z[-1][:, :1].detach()) > 0.5).float(),
#                     self.nonlinearity_norm(agg_z[-1][:, 1:]),
#                     x[
#                         :, 
#                         :, 
#                         self.closing_size * (i+1):-self.closing_size * (i+1),
#                         self.closing_size * (i+1):-self.closing_size * (i+1),
#                         self.closing_size * (i+1):-self.closing_size * (i+1),
#                     ]
#                 ], dim=1)
#             )
#             agg_z.append(z)
#         return agg_z

#     def masks(self, y: torch.Tensor) -> torch.Tensor:
#         agg_mask = []
#         for i in range(len(self.layers)):
#             agg_mask.append(
#                 y[
#                     :,
#                     :,
#                     self.closing_size * (i+1):-self.closing_size * (i+1),
#                     self.closing_size * (i+1):-self.closing_size * (i+1),
#                     self.closing_size * (i+1):-self.closing_size * (i+1),
#                 ]
#             )
#         return agg_mask

# batch_size = 8
# train_data = DataLoader(train, batch_size=batch_size, shuffle=True)
# model = _FilterNet().to(device)

# x, y, _ = next(iter(train_data))
# aug = util.PatchAugment().to(device)
# x, y = aug(x.float().to(device)), aug(y.float().to(device))

# unproc_pred = x[:, :1, 32:-32, 32:-32, 32:-32].contiguous()
# true_mask = y[..., 32:-32, 32:-32, 32:-32].contiguous()
# # x, y = F.max_pool3d(x, 4), F.max_pool3d(y, 4)


# # Compute output

# p_y = [F.sigmoid(logits) for logits in model(x)]
# proc_pred = [(p > 0.5).float() for p in p_y]
# masks = model.masks(y)

# [p.shape for p in p_y], [m.shape for m in masks]

In [4]:
class FilterNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # assert len(layers) >= 2, "layers must have at least 2 elements"

        kernel_size = 9
        layers = [2, 2, 2, 2, 2,2,2,2, 3]

        c = layers[0]
        # L = []
        self.layers = nn.ModuleList()
        for i, l in enumerate(layers[1:-1]):
            # L.append(util.FFTConv3d(c, l, kernel_size, padding=0 if i < 2 else kernel_size//2))
            # L.append(torch.nn.BatchNorm3d(l))
            # L.append(torch.nn.GELU())
            # c = l
            self.layers.append(nn.Sequential(
                util.FFTConv3d(c, l, kernel_size),#, padding=0 if i < 2 else kernel_size//2),
                # torch.nn.BatchNorm3d(l),
                torch.nn.GELU(),
            ))
            c = l
        self.layers.append(
            util.FFTConv3d(c, layers[-1], kernel_size)#, padding=0 if i < 2 else kernel_size//2),
            # torch.nn.BatchNorm3d(layers[-1]),
        )
    
        # self.net = torch.nn.Sequential(*L)
    
    def forward(self, x):
        z = self.layers[0](x)
        for layer in self.layers[1:]:
            z = layer(z)
        return z
        # z = self.layers[0](x)
        # z = self.layers[1](z) #+ (x[..., 32:-32, 32:-32, 32:-32])
        # z = self.layers[2](z)
        # z = self.layers[3](z) #+ z
        # z = self.layers[4](z) #+ (x[:, :1, 32:-32, 32:-32, 32:-32] - 0.5)
        # return z

        # for layer in self.layers[2:]:
        #     z = layer(z) + z
        # return z + (x[..., 32:-32, 32:-32, 32:-32] - 0.5)

# model = FilterNet().to(device)
# x = torch.rand(16, 2, 128, 128, 128).to(device)
# y = model(x)
# y.shape

In [5]:
log_board.clear('train')
log_board.clear('val')

In [6]:

save_every = 1000
epochs = 1000
batch_size = 32
t_logger = log_board.get_logger('train')
v_logger = log_board.get_logger('val')

train_data = DataLoader(train, batch_size=batch_size, shuffle=True)
# valid_data = DataLoader(test, batch_size=batch_size, shuffle=True)

# F.max_pool3d()

# model = util.UNet3D(
#     layers=[16, 32, 64, 64, 128, 128, 256],
#     Conv3d=util.nn.Conv3DNormed,
#     block_depth=8,
#     connect_depth=4,
#     dropout=0.1,
# ).to(device)
# model.load_state_dict(torch.load('./bin/models/unet3d.pt', map_location=device))
# model.train()


model = FilterNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
dice_fn = util.DiceScore().to(device)
focal_loss = util.BinaryFocalLoss(
    alpha=0.5,
    gamma=1.5,
)

bce = torch.nn.MSELoss()


t = time()
step = 0
for epoch in range(epochs):
    for i, (x, y, _) in enumerate(train_data):
        step += 1

        # Prepare data
        aug = util.PatchAugment().to(device)
        x, y = aug(x.float().to(device)), aug(y.float().to(device))

        unproc_pred = x[:, :1, 32:-32, 32:-32, 32:-32].contiguous()
        true_mask = y[..., 32:-32, 32:-32, 32:-32].contiguous()

        FP_mask = ((true_mask == 0) & (unproc_pred == 1)).float()
        FN_mask = ((true_mask == 1) & (unproc_pred == 0)).float()
        # x, y = F.max_pool3d(x, 4), F.max_pool3d(y, 4)


        # Compute output
        p_y = F.sigmoid(model(x))
        pred_mask = (p_y > 0.5).float()

        # # Compute loss
        # unprocessed_loss = 1- dice_fn(unproc_pred, true_mask)
        # processed_loss = 1 - dice_fn(p_y, true_mask)
        # loss = processed_loss - unprocessed_loss

        fp_dice_loss = 1 - dice_fn(p_y[:, 1].unsqueeze(1).contiguous(), FP_mask)
        fn_dice_loss = 1 - dice_fn(p_y[:, 0].unsqueeze(1).contiguous(), FN_mask)
        loss = fp_dice_loss + fn_dice_loss

        # Step grad
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 1)
        optimizer.step()
        optimizer.zero_grad()

        # Logging & validation
        # if (step + 1) % 10 == 0:
        with torch.no_grad():
            # log loss
            t_logger.add_scalar('loss', loss, step)

            final_mask = unproc_pred + pred_mask[:, 1].unsqueeze(1) - pred_mask[:, 0].unsqueeze(1)
            final_mask = torch.clamp(final_mask, 0, 1)
            t_logger.add_scalar('dice', dice_fn(final_mask, true_mask), step)
            t_logger.add_scalar('acc', torch.eq(final_mask, true_mask).float().mean(), step)

            # stats
            t_logger.add_scalars(f'stats',{
                # 'prob_std': p_y.std(),
                # 'prob_mean': p_y.mean(),
                'pred_std': final_mask.std(),
                'pred_mean': final_mask.mean(),
                'mask_std': true_mask.std(),
                'mask_mean': true_mask.mean(),
            }, step)

            # # Validation logging
            #     model.eval()
            #     x, y, _ = next(iter(valid_data))
            #     x, y = x.float().to(device), y.float().to(device)
            #     # x, y = F.max_pool3d(x, 4), F.max_pool3d(y, 4)

            #     # Compute output
            #     p_y = [F.sigmoid(logits) for logits in model(x, up_depth=2)]
            #     pred_masks = [(p > 0.5).float() for p in p_y]
            #     masks = model.deep_masks(y)[1:3]

            #     acc = [torch.eq(pred_masks[i], masks[i]).float().mean() for i in dlayers]
            #     dice = [dice_fn(pred_masks[i], masks[i]) for i in dlayers]
            #     perf_dict = { f'acc_{k}' : acc[i] for i, k in enumerate(dlayers) }
            #     perf_dict.update({ f'dice_{k}' : dice[i] for i, k in enumerate(dlayers) })
            #     v_logger.add_scalars('performance', perf_dict, step)

            #     v_logger.add_image('masks', util.mask_plots(x, masks, pred_masks), step, dataformats='HWC')
            #     t_logger.add_scalar('time', time() - t, step)
            #     t = time()
            #     model.train()
                
            # # Save model
            # if (step + 1) % save_every == 0:
            #     torch.save(model.state_dict(), f'./bin/_tmp_models/wide_patcher_finder_IN_PROGRESS.pt')
            if (step + 1) % 10 == 0:
                t_logger.add_image('masks', util.mask_plots(x[:, 1:, 32:-32, 32:-32, 32:-32], [true_mask, true_mask], [unproc_pred, final_mask]), step, dataformats='HWC')
                t_logger.add_scalar('time', time() - t, step)



KeyboardInterrupt: 

In [1]:
128/8

16.0

In [6]:
import os

os.makedirs('./bin/models', exist_ok=True)

torch.save(model.state_dict(), f'./bin/models/unet3d2.pt')

In [6]:
import torch
x = torch.rand(32, 1, 16, 16, 16)
x.amax((2,3,4)).shape

torch.Size([32, 1])